In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
hospital_file = "../Resources/hospital_visits.csv"
hospital_df = pd.read_csv(hospital_file)
hospital_df.head()

,FIPS,3 / 1 / 2020,3 / 2 / 2020,3 / 3 / 2020,3 / 4 / 2020,3 / 5 / 2020,3 / 6 / 2020,3 / 7 / 2020,3 / 8 / 2020,3 / 9 / 2020,...,3 / 12 / 2020,3 / 13 / 2020,3 / 14 / 2020,3 / 15 / 2020,3 / 16 / 2020,3 / 17 / 2020,3 / 18 / 2020,3 / 19 / 2020,3 / 20 / 2020,3 / 21 / 2020
0,1001,80.0,62.0,80.0,62.0,73.0,85.0,91.0,73.0,62.0,...,70.0,62.0,43.0,42.0,48.0,65.0,48.0,47.0,38.0,31.0
1,1003,203.0,469.0,477.0,474.0,451.0,465.0,243.0,225.0,486.0,...,454.0,391.0,168.0,163.0,364.0,339.0,331.0,383.0,272.0,106.0
2,1005,33.0,61.0,70.0,36.0,63.0,42.0,21.0,23.0,43.0,...,44.0,31.0,22.0,11.0,41.0,48.0,52.0,38.0,27.0,8.0
3,1007,62.0,77.0,83.0,69.0,62.0,59.0,55.0,40.0,74.0,...,83.0,55.0,43.0,32.0,64.0,65.0,55.0,52.0,41.0,17.0
4,1009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
county_file = "../Resources/us_county.csv"
county_df = pd.read_csv(county_file)
county_df.head()

,fips,county,state,state_code,male,female,median_age,population,female_percentage,lat,long
0,1001,Autauga County,Alabama,AL,26874,28326,37.8,55200,51.315217,32.534923,-86.642730
1,1003,Baldwin County,Alabama,AL,101188,106919,42.8,208107,51.376936,30.727479,-87.722564
2,1005,Barbour County,Alabama,AL,13697,12085,39.9,25782,46.873788,31.869581,-85.393210
3,1007,Bibb County,Alabama,AL,12152,10375,39.9,22527,46.055844,32.998628,-87.126475
4,1009,Blount County,Alabama,AL,28434,29211,40.8,57645,50.673953,33.980869,-86.567380


### Transform hospital DataFrame

In [4]:
# Create a filtered dataframe from specific columns
hospital_transformed_df = hospital_df.melt(id_vars=["FIPS"], 
        var_name="Date", 
        value_name="Visits")

hospital_transformed_df = hospital_transformed_df.rename(columns={"FIPS":"fips",
                                                                 "Date":"date",
                                                                 "Visits":"visits"})
#hospital_transformed_df.drop_duplicates("id", inplace = True)
hospital_transformed_df.reset_index()

hospital_transformed_df.head(10)

,fips,date,visits
0,1001,3 / 1 / 2020,80.0
1,1003,3 / 1 / 2020,203.0
2,1005,3 / 1 / 2020,33.0
3,1007,3 / 1 / 2020,62.0
4,1009,3 / 1 / 2020,0.0
5,1011,3 / 1 / 2020,5.0
6,1013,3 / 1 / 2020,19.0
7,1015,3 / 1 / 2020,0.0
8,1017,3 / 1 / 2020,0.0
9,1019,3 / 1 / 2020,12.0


### Transform county DataFrame

In [5]:
county_cols = ["fips","county","state_code","male","female","median_age", "population"]
county_transformed = county_df[county_cols].copy()

# Rename the column headers
county_transformed.drop_duplicates("fips", inplace = True)
county_transformed.reset_index
county_transformed.head()

,fips,county,state_code,male,female,median_age,population
0,1001,Autauga County,AL,26874,28326,37.8,55200
1,1003,Baldwin County,AL,101188,106919,42.8,208107
2,1005,Barbour County,AL,13697,12085,39.9,25782
3,1007,Bibb County,AL,12152,10375,39.9,22527
4,1009,Blount County,AL,28434,29211,40.8,57645


### Create database connection

In [6]:
connection_string = "postgres:Rodent6611@localhost:5432/covid_counties"
engine = create_engine(f'postgresql://{connection_string}')

In [7]:
# Confirm tables
engine.table_names()

['hospital_visits', 'us_county']

### Load DataFrames into database

In [8]:
hospital_transformed_df.to_sql(name='hospital_visits', con=engine, if_exists='append', index_label = 'id')

In [9]:
county_transformed.to_sql(name='us_county', con=engine, if_exists='append', index_label ='id')